In [ ]:
import pandas as pd

df = pd.DataFrame().from_dict({'X': [0,1,2,28,18,30], 'Y': [0,0,0,1,1,1]})
df

## Let's visulize it

In [ ]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Slider, Div
from bokeh.plotting import figure, output_notebook, show
from bokeh.themes import built_in_themes
from bokeh.io import curdoc

output_notebook()
curdoc().theme = 'night_sky'

source = ColumnDataSource(df)

plot = figure(plot_width=400, plot_height=400)
plot.circle(x='X', y='Y', source=source, size=10, color="green", alpha=0.5)

show(plot)

# Slider Game (again?)

In [ ]:
import math
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Slider, Div
from bokeh.plotting import figure, output_notebook, show
from bokeh.themes import built_in_themes
from bokeh.io import curdoc

output_notebook()
curdoc().theme = 'night_sky'

def sigmiod_func(df):
    return 1/(1+math.exp(-1*(df.b1*df.X+df.b0)))

def cost_func(df):
    y = list(df['Y'])
    y_pred = list(df['Y pred'])
    sum = 0
    
    for idx in range(len(y)):
        sum += -y[idx]*math.log(y_pred[idx]) - (1-y[idx])*math.log(1-y_pred[idx])
    return sum

b1 = 0.425
b0 = 0.785

df = pd.DataFrame().from_dict({'X': [0,1,2,28,18,30], 'Y': [0,0,0,1,1,1]})
df['b1'] = b1
df['b0'] = b0
df['Y pred'] = df.apply(sigmiod_func, axis=1)

line_df = pd.DataFrame().from_dict({'X': [x*0.05 for x in range(0, 600)]})
line_df['b1'] = b1
line_df['b0'] = b0
line_df['Y'] = line_df.apply(sigmiod_func, axis=1)

source = ColumnDataSource(df)
line_source = ColumnDataSource(line_df)

plot = figure(plot_width=400, plot_height=400, x_range=[-5,35], y_range=[-0.5,1.5])
plot.line(x='X', y='Y', source=line_source, line_width=3, line_alpha=0.6)
plot.circle(x='X', y='Y', source=source, size=10, color="green", alpha=0.5)

callback_b1 = CustomJS(args=dict(source=line_source), code="""
        var data = source.data;
        var b1_val = cb_obj.value
        var b1 = data['b1']
        var b0 = data['b0']
        var x = data['X']
        var y = data['Y']
        for (var i = 0; i < x.length; i++) {
            b1[i] = b1_val
            y[i] = 1.0/(1+Math.exp(-1*(b1[i]*x[i]+b0[i])))
        }
        source.change.emit();
    """)

callback_b0 = CustomJS(args=dict(source=line_source), code="""
        var data = source.data;
        var b0_val = cb_obj.value
        var b1 = data['b1']
        var b0 = data['b0']
        var x = data['X']
        var y = data['Y']
        for (var i = 0; i < x.length; i++) {
            b0[i] = b0_val
            y[i] = 1.0/(1+Math.exp(-1*(b1[i]*x[i]+b0[i])))
        }
        source.change.emit();
    """)

div = Div(text="Cost function: "+ str(cost_func(source.data)))

change_text_b1 = CustomJS(args=dict(div=div, source=source), code="""
                var data = source.data;
                var b1_val = cb_obj.value
                var b1 = data['b1']
                var b0 = data['b0']
                var x = data['X']
                var y_pred = data['Y pred'];
                var y = data['Y'];
                var sum = 0;
                for (var i = 0; i < y.length; i++) {
                    b1[i] = b1_val
                    y_pred[i] = 1.0/(1+Math.exp(-1*(b1[i]*x[i]+b0[i])))
                    sum += -y[i]*Math.log(y_pred[i]) - (1-y[i])*Math.log(1-y_pred[i])
                }
                div.text = "Cost function: " + sum;
             """       
    )

change_text_b0 = CustomJS(args=dict(div=div, source=source), code="""
                var data = source.data;
                var b0_val = cb_obj.value
                var b1 = data['b1']
                var b0 = data['b0']
                var x = data['X']
                var y_pred = data['Y pred'];
                var y = data['Y'];
                var sum = 0;
                for (var i = 0; i < y.length; i++) {
                    b0[i] = b0_val
                    y_pred[i] = 1.0/(1+Math.exp(-1*(b1[i]*x[i]+b0[i])))
                    sum += -y[i]*Math.log(y_pred[i]) - (1-y[i])*Math.log(1-y_pred[i])
                }
                div.text = "Cost function: " + sum;
             """       
    )

slider_b1 = Slider(start=0, end=4, value=b1, step=.1, title="value of b1")
slider_b1.js_on_change('value', callback_b1, change_text_b1)
slider_b0 = Slider(start=-10, end=4, value=b0, step=.1, title="value of b0")
slider_b0.js_on_change('value', callback_b0, change_text_b0)

layout = column(slider_b1, slider_b0, plot, div)

show(layout)

In [ ]:
line_df.tail()

### Predicting with scikit learn
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
import numpy as np

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0)
clf.fit(df["X"].values.reshape(-1, 1), df["Y"].values)


In [ ]:
clf.predict(np.array(12).reshape(-1, 1))

In [ ]:
clf.predict_proba(np.array(10).reshape(-1, 1))